In [2]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import os
import json
import cv2

# Load mango model
model_path = os.path.join(r"C:\Users\Pavan\Pavaman Tech\disease_detection\seperate_models\Model\Mango_model\cnn_mango_disease_experiments\mango_leaf_disease_model.keras")
model = load_model(model_path)

# Load class names
with open(os.path.join("data", "mango_class_names.json"), "r") as f:
    class_names = json.load(f)

# Preprocess mango image like in training
def preprocess_mango_image(img_pil, target_size=(224, 224)):
    img = np.array(img_pil)
    img = cv2.resize(img, (256, 256))

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower = np.array([25, 40, 40])
    upper = np.array([95, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    result = cv2.bitwise_and(img, img, mask=mask)

    lab = cv2.cvtColor(result, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    final = cv2.resize(enhanced, target_size)
    return final / 255.0

# Load test image
img_path = r"C:\Users\Pavan\Pavaman Tech\disease_detection\seperate_models\Data\open_source_mango_data\Bacterial_Canker\IMG_20240504_152005.jpg"
img_pil = Image.open(img_path).convert("RGB")
processed = np.expand_dims(preprocess_mango_image(img_pil), axis=0)

# Predict
pred = model.predict(processed)[0]
idx = int(np.argmax(pred))
label = class_names[str(idx)]
conf = float(np.max(pred))

print(f"[TEST] Predicted: {label} ({conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
[TEST] Predicted: Mango - Bacterial Canker (0.99)
